In [1]:
from ipyleaflet import * 
import time

import pandas as pd
import seaborn as sns
import scipy as sp
import sklearn as sk
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D


import numpy as np
import scipy as sp
import sklearn as sk

mpl.use('Agg')

%matplotlib inline

sns.set()

from dateutil import parser

import folium

train_raw = pd.read_csv("train.csv", parse_dates = True)

data = train_raw.copy()

C:\Users\ledes\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\ledes\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\ledes\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\ledes\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ledes\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\ledes\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\U

### 명확한 아웃라이어 제거(현실성 판단)
1. trip_duration 기준  
2. passenger_count 기준  
3. lat, long 기준  

#### 시각화를 통한 데이터 셋 확인

In [ ]:
sns.boxplot(y = data["trip_duration"], x = data["passenger_count"], data = data)
plt.show()

In [21]:
#trip_duration > 1500000 확인
over_million = data[data["trip_duration"] > 1500000]
pick_lat, pick_long = over_million["pickup_latitude"], over_million["pickup_longitude"]
drop_lat, drop_long = over_million["dropoff_latitude"], over_million["dropoff_longitude"]
info = over_million["id"]

center = [40.745219,-73.9935417] # lat, long
map_osm = folium.Map(location = center, zoom_start = 10, tiles = 'Stamen toner') # Stamen toner - 빠른 로딩 가능 
map_osm

for lat, long, info_ in zip(pick_lat, pick_long, info) :
    print(lat, long)
    folium.Marker([lat, long], popup = info_ , icon = folium.Icon(color = 'blue')).add_to(map_osm)
for lat, long, info_ in zip(drop_lat, drop_long, info) :
    folium.Marker([lat, long], popup = info_ , icon = folium.Icon(color = 'red')).add_to(map_osm)
    
map_osm

40.64355850219727 -73.7896499633789
40.735252380371094 -73.92167663574219
40.74232482910156 -73.9837875366211
40.64863204956055 -73.78390502929686


4개의 데이터 승하차 지역 대비 이동시간이 너무 김 -> Human Error로 보이므로 제거

In [ ]:
data = data[data["trip_duration"] < 1500000].reset_index(drop = True)

In [ ]:
#승객 수 0명 제거
data = data[data["passenger_count"] != 0 ]

In [35]:
#승객 수 7,8,9명 데이터 확인
over_seven = data[(data.passenger_count >= 7) & (data.trip_duration > 60)]
over_seven

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
436047,id2489742,2,2016-06-24 08:09:21,2016-06-24 08:18:41,9,-73.710632,40.671581,-73.710632,40.671581,N,560
826064,id1488553,2,2016-01-01 01:15:20,2016-01-01 01:17:04,8,-73.992653,40.768719,-74.041374,40.729954,N,104


In [39]:
#승객수 7,8,9 명 & 탑승시간 1분 미만 제거, 승하차 지점 확인
over_seven = data[(data["passenger_count"] >= 7 ) & (data["trip_duration"] > 60)]
pick_lat, pick_long = over_seven["pickup_latitude"], over_seven["pickup_longitude"]
drop_lat, drop_long = over_seven["dropoff_latitude"], over_seven["dropoff_longitude"]
info = over_seven["id"]

center = [40.745219,-73.9935417] # lat, long
map2 = folium.Map(location = center, zoom_start = 10, tiles = 'Stamen toner') # Stamen toner - 빠른 로딩 가능 
map2

for lat, long, info_ in zip(pick_lat, pick_long, info) :
    folium.Marker([lat, long], popup = info_ , icon = folium.Icon(color = 'blue')).add_to(map2)
for lat, long, info_ in zip(drop_lat, drop_long, info) :
    folium.Marker([lat, long], popup = info_ , icon = folium.Icon(color = 'red')).add_to(map2)
    
map2


데이터 수가 적다고 해도, 일일히 확인하며 제거할 수는 없음  
저녁 먹고 leverge 분석 공부할 것.